In [47]:
!git clone https://github.com/SUJITHhubpost/glaucomaDetection

fatal: destination path 'glaucomaDetection' already exists and is not an empty directory.


In [0]:


# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.models import load_model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
from keras.optimizers import SGD
from glaucomaDetection.config.resnet import ResNet
from glaucomaDetection.config import config
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import os


In [0]:

# initialize the path to the *original* input directory of images
ORIG_INPUT_DATASET = "glaucomaDetection/glaucoma/data"

# initialize the base path to the *new* directory that will contain
# our images after computing the training and testing split
BASE_PATH = "glaucomaDetection/glaucoma"

# derive the training, validation, and testing directories
TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

# define the amount of data that will be used training
TRAIN_SPLIT = 0.7

# the amount of validation data will be a percentage of the
# *training* data
VAL_SPLIT = 0.2


In [0]:


# define the total number of epochs to train for along with the
# initial learning rate and batch size
NUM_EPOCHS = 5000
INIT_LR = 1e-2
BS = 32



In [0]:
def poly_decay(epoch):
	# initialize the maximum number of epochs, base learning rate,
	# and power of the polynomial
	maxEpochs = NUM_EPOCHS
	baseLR = INIT_LR
	power = 1.0

	# compute the new learning rate based on polynomial decay
	alpha = baseLR * (1 - (epoch / float(maxEpochs))) ** power

	# return the new learning rate
	return alpha


In [0]:

# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(config.TRAIN_PATH)))
totalVal = len(list(paths.list_images(config.VAL_PATH)))
totalTest = len(list(paths.list_images(config.TEST_PATH)))


In [0]:

# initialize the training training data augmentation object
trainAug = ImageDataGenerator(
	rescale=1 / 255.0,
	rotation_range=20,
	zoom_range=0.05,
	width_shift_range=0.05,
	height_shift_range=0.05,
	shear_range=0.05,
	horizontal_flip=True,
	fill_mode="nearest")


In [122]:

# initialize the validation (and testing) data augmentation object
valAug = ImageDataGenerator(rescale=1 / 255.0)

# initialize the training generator
trainGen = trainAug.flow_from_directory(
	TRAIN_PATH,
	class_mode="categorical",
	target_size=(256, 256),
	color_mode="rgb",
	shuffle=True,
	batch_size=BS)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	VAL_PATH,
	class_mode="categorical",
	target_size=(256, 256),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	TEST_PATH,
	class_mode="categorical",
	target_size=(256, 256),
	color_mode="rgb",
	shuffle=False,
	batch_size=BS)

Found 130 images belonging to 2 classes.
Found 32 images belonging to 2 classes.
Found 32 images belonging to 2 classes.


In [0]:


# initialize our ResNet model and compile it
model = ResNet.build(256, 256, 3, 2, (3, 4, 6),
	(256, 128, 256, 512), reg=0.0005)
opt = SGD(lr=INIT_LR, momentum=0.9)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])


In [124]:

# define our set of callbacks and fit the model
callbacks = [LearningRateScheduler(poly_decay)]

print("[INFO] Training network...")

H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // BS,
	validation_data=valGen,
	validation_steps=totalVal // BS,
	epochs=NUM_EPOCHS,
  verbose = 0,
	callbacks=callbacks)

print("[INFO] Training Done...")

# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // BS) + 1)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))



[INFO] Training network...
[INFO] Training Done...
[INFO] evaluating network...
              precision    recall  f1-score   support

Glaucomatous       0.62      1.00      0.77        18
      normal       1.00      0.21      0.35        14

    accuracy                           0.66        32
   macro avg       0.81      0.61      0.56        32
weighted avg       0.79      0.66      0.59        32



In [125]:

# serialize the model to disk
print("[INFO] saving detector model...")
model.save("model.h5")
print("[INFO] model Saved")


[INFO] saving detector model...
[INFO] model Saved


In [126]:
print(H.history)

{}


In [127]:

# plot the training loss and accuracy
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig("plot.png")


KeyError: ignored

In [0]:


def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(256, 256))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor = 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor




In [135]:
# load model
print("[INFO] Model Loading ...")
model = load_model("model.h5")
print("[INFO] Model Loaded")

# image path
img_path = 'glaucomaDetection/glaucoma/data/Glaucomatous/drishtiGS_050.png'    # normal
#img_path = 'glaucomaDetection/glaucoma/data/Glaucomatous/1ffa9334-8d87-11e8-9daf-6045cb817f5b..JPG'      # glaucoma positive

# load a single image
new_image = load_image(img_path)

# check prediction
pred = model.predict(new_image)
print(pred)

[INFO] Model Loading ...
[INFO] Model Loaded


AttributeError: ignored